<a href="https://colab.research.google.com/github/Kavia-M/Deep-Learning/blob/main/CNN_for_CIFAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

# Load dataset CIFAR10

In [2]:
from keras.datasets import cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

170500096/170498071 [==============================] - 3s 0us/step


In [3]:
print('Train data shape', train_images.shape , train_labels.shape)
print('Test data shape', test_images.shape, test_labels.shape)

Train data shape (50000, 32, 32, 3) (50000, 1)
Test data shape (10000, 32, 32, 3) (10000, 1)


size of each data is 32 * 32 * 3 (3 means color image)

In [4]:
classes = np.unique(train_labels)   #use this to find the diffent classes in train data so that we can use same number of neurons in softmax layer
nClasses = len(classes)

print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

plt.figure(figsize=[4,2])

Total number of outputs :  10
Output classes :  [0 1 2 3 4 5 6 7 8 9]


<Figure size 288x144 with 0 Axes>

<Figure size 288x144 with 0 Axes>

# Preproceess data

In [6]:
nRows, nCols, nDims = train_images.shape[1:] # (50000, 32 , 32, 3)--> we take from index 1 that is nRows =32, nCols =32, nDims=3
train_data = train_images.reshape(train_images.shape[0], nRows, nCols, nDims) 
test_data = test_images.reshape(test_images.shape[0], nRows, nCols, nDims)
input_shape = (nRows, nCols, nDims)

train_data = train_data.astype('float32')
test_data = test_data.astype('float32')

train_data /= 255   # data to range of 0 to 1
test_data /=255

train_labels_one_hot = to_categorical(train_labels) # very important...one hot matrix has 10 cells, if categor is 6 ..index 6 element is 1 and others are 0
test_labels_one_hot = to_categorical(test_labels)

print('Original label 0 : ', train_labels[0])
print('After convertion to categorical (one hot) : ', train_labels_one_hot[0])

Original label 0 :  [6]
After convertion to categorical (one hot) :  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


In [7]:
train_images.shape[0]

50000

# Define CNN model

models are usually sequential because the layers are palced one after another


In [8]:
model = Sequential()

In [9]:
# first 2 layers with 32 filters of window size 3x3
model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=input_shape)) #32 channels 3x3 kernel size
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), padding='same', activation='relu')) #one more convolutional layer with 64 channels
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(512, activation='relu')) #FULLY connected layer
model.add(Dropout(0.5)) #regularization only 50% takes place in classification decision
model.add(Dense(nClasses, activation='softmax')) #output layer with softmax activation...number of neurons is nClasses 

# Compile

In [10]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])  #adam optimiser can be also used...
#categorical cross entropy since number of classes is more than 2

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 4096)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               2097664   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0

(notes) droput layer has 512  neurons but 50% only gets activated
all are trainable parameters

In [15]:
batch_size = 128
epochs = 2

In [16]:
#execute the model  ... verbose=1 will show you an animated progress bar like this: [===================]
history = model.fit(train_data, train_labels_one_hot, batch_size=batch_size, epochs=epochs, verbose=1,  
                    validation_data=(test_data, test_labels_one_hot))  #validation is taken test data itself
model.evaluate(test_data, test_labels_one_hot)  #testing

Epoch 1/2
391/391 [==============================] - 86s 217ms/step - loss: 1.8737 - accuracy: 0.3307 - val_loss: 1.2458 - val_accuracy: 0.5582
Epoch 2/2
313/313 [==============================] - 5s 17ms/step - loss: 1.0661 - accuracy: 0.6264


[1.0660653114318848, 0.6263999938964844]

(notes)...accuracy is very low in testing sinec we ran only 2 epochs

predefined alexNet LeNet and all will have too many conv and pooling layers...
this above 2 epoch takes more time since all are iamges
100 epochs would run for 1 day

INCREASED number of layers will take more time